In [21]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, QISKitError
from qiskit import available_backends, register, execute, get_backend
from qiskit.tools.visualization import circuit_drawer, plot_histogram, plot_state
from qiskit.tools.qi.qi import state_fidelity
import random
import pickle
import time
import datetime
import xlsxwriter

#setup
import sys, getpass
try:
    sys.path.append("../../") # go to parent dir
    import Qconfig
    qx_config = {
        "APItoken": Qconfig.APItoken,
        "url": Qconfig.config['url']}
    print('Qconfig loaded from %s.' % Qconfig.__file__)
except:
    APItoken = getpass.getpass('Please input your token and hit enter: ')
    qx_config = {
        "APItoken": APItoken,
        "url":"https://quantumexperience.ng.bluemix.net/api"}
    print('Qconfig.py not found in qiskit-tutorial directory; Qconfig loaded using user input.')
    
#matlab stuff
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from numpy import matrix
from math import pi

#set api token 
QX_TOKEN = "36ad8b31280d5d431c22e75d77858b3500186b7a4836b0b25199fe40edf502addfa35cb19ab8d1bd91e67418c2039efb5f7310bae262bee1a114c6523e7aee4a"
QX_URL = "https://quantumexperience.ng.bluemix.net/api"


#register token and url
register(QX_TOKEN, QX_URL)

backend = 'ibmqx5'

# prepare a state of all zeros
def prepareZeros(qc):
    qc.h(qr[1])
    qc.barrier()
    qc.cx(qr[1],qr[0])
    qc.barrier()
    qc.cx(qr[1],qr[2])
    qc.barrier()
    qc.cx(qr[2],qr[3])
    qc.barrier()

    
# compile into different states
def compilation(qc,i,perm):
    if i == 0: 
        return
    elif i == 1:
        qc.x(qr[perm[0]])
        qc.x(qr[perm[2]])
    elif i == 2:
        qc.x(qr[perm[0]])
        qc.x(qr[perm[1]])
    elif i == 3:
        qc.x(qr[perm[2]])
        qc.x(qr[perm[1]])
    qc.barrier()
            
# add phase gate
def addPhase(qc):
    qc.h(qr[0])
    qc.h(qr[1])
    qc.h(qr[2])
    qc.h(qr[3])
    qc.barrier()
    qc.cx(qr[1],qr[0])
    qc.barrier()
    qc.s(qr[0])
    qc.barrier()
    qc.cx(qr[1],qr[0])
    qc.barrier()
    
# remove phase gate
def removePhase(qc,perm):
    qc.cx(qr[perm[1]], qr[perm[0]])
    qc.barrier()
    qc.sdg(qr[perm[0]])
    qc.barrier()
    qc.cx(qr[perm[1]], qr[perm[0]])
    qc.barrier()
    qc.h(qr[3])
    qc.h(qr[2])
    qc.h(qr[1])
    qc.h(qr[0])
    qc.barrier()
    
# parse a string of gens
def addCliffGens(qc, string, perm):
    for char in string:
        i = int(char)
        if i==0:
            qc.x(qr[perm[0]])
            qc.x(qr[perm[2]])
        elif i==1:
            qc.x(qr[perm[0]])
            qc.x(qr[perm[1]])
        elif i==2:
            qc.z(qr[perm[0]])
            qc.z(qr[perm[1]])
        elif i==3:
            qc.z(qr[perm[0]])
            qc.z(qr[perm[2]])
        elif i ==4:
            qc.h(qr[0])
            qc.h(qr[1])
            qc.h(qr[2])
            qc.h(qr[3])
        elif i == 5:
            qc.s(qr[0])
            qc.s(qr[1])
            qc.s(qr[2])
            qc.s(qr[3])
            qc.barrier()
        elif i==6:
            swap(perm,0,1)
        elif i==7:
            swap(perm,0,2)        

# find string for generators
def findGenString(mat):
    for key, value in signedGroup.items():
        if np.array_equal(value, mat):
            return key
    print("OH NO")
    return None
        
# invert cliff gens 
def invertCliffGens(qc, genString, perm):
    for char in genString[::-1]:
        i = int(char)
        if i==0:
            qc.x(qr[perm[0]])
            qc.x(qr[perm[2]])
        elif i==1:
            qc.x(qr[perm[0]])
            qc.x(qr[perm[1]])
        elif i==2:
            qc.z(qr[perm[0]])
            qc.z(qr[perm[1]])
        elif i==3:
            qc.z(qr[perm[0]])
            qc.z(qr[perm[2]])
        elif i ==4:
            qc.h(qr[0])
            qc.h(qr[1])
            qc.h(qr[2])
            qc.h(qr[3])
        elif i == 5:
            qc.sdg(qr[0])
            qc.sdg(qr[1])
            qc.sdg(qr[2])
            qc.sdg(qr[3])
        elif i==6:
            swap(perm,0,1)
        elif i==7:
            swap(perm,0,2)
    qc.barrier()

# adds random real clifford
def addRandCliff(qc, mat,perm):
    genString = random.choice(list(r2group.keys()))
    addCliffGens(qc,genString,perm)
    qc.barrier()
    return np.matmul(mat,r2group[genString])

# used for reindexing qubits
def swap(perm, a, b):
    temp = perm[a]
    perm[a] = perm[b]
    perm[b] = temp
    
# converts 0-3 to binary strings
def toBinString(num):
    if num ==0:
        return "00"
    elif num == 1:
        return "01"
    elif num == 2:
        return "10"
    elif num == 3:
        return "11"
    else:
        print("OH NO")

def printDataFormatted(data):
    strings = ["0000", "1111", "1010", "0101", "0011", "1100", "1001", "0110", "1000", "0111", "0001", "1110"
              , "0010", "1101", "0100", "1011"]
    for string in strings:
        try:
            print(string+": "+str(data[string]))
        except:
            print("oops, none of "+string)
            pass
    

def writeToExcel(results,key,initialState,row,col):
    states = ["0000", "1111", "1010", "0101", "0011", "1100", "1001", "0110", "1000", "0111", "0001", "1110"
              , "0010", "1101", "0100", "1011"]
    print(results)
    worksheet.write(row,0,initialState)
    worksheet.write(row,1,key)
    col = 2
    for state in states:
        try:
            worksheet.write(row,col,results[state])
            
        except:
            print(state+"does not exist")
            worksheet.write(row,col,0)
        col = col+1
    
    return row+1
        
        
lengths = [3]
reps = [9,9,9,9]
initialStates = [0,1,2,3]
shots = 1024
phased = False
shouldWrite = True

device = get_backend("ibmqx5")
coupling_map = device.configuration['coupling_map']

pickle_in = open("r2group.pkl", "rb")
r2group = pickle.load(pickle_in)
pickle_in2 = open("r2group.pickle", "rb")
signedGroup = pickle.load(pickle_in2)

if shouldWrite:
    workbook = xlsxwriter.Workbook("encoded_("+
                                   ",".join(map(str,lengths))+
                                   ")_("+
                                   ",".join(map(str,reps))+
                                   ")_"+
                                   str(int(phased))+
                                   "_"+
                                   str(datetime.datetime.now().strftime('%Y%m%d_%H%M%S'))+
                                   ".xlsx")
    worksheet = workbook.add_worksheet()
    
    states = ["Key","0000", "1111", "1010", "0101", "0011", "1100", "1001", "0110", "1000", "0111", "0001", "1110"
              , "0010", "1101", "0100", "1011"]
    for index in range(len(states)):
        worksheet.write(0,index+1,states[index])
    row = 1
    col = 1

startTime = time.time()

for length in lengths:

    for compiler in initialStates:
        
        if shouldWrite:
            worksheet.write(row,0, "Length: "+str(length))
            row = row+1
            
        print(str(length)+" "+str(compiler)+"\n")

        # run our trials
        for i in range(reps[compiler]):

            # run one trial of given length 
            mat = r2group[""]
            n = 4
            qr = QuantumRegister(n)
            cr = ClassicalRegister(n)
            qc = QuantumCircuit(qr, cr)
            
            perm = {0: 0, 1: 1, 2: 2, 3: 3}
            
            # prepare encoded zeros
            prepareZeros(qc) 
            
            # optional: add phase
            if phased:
                addPhase(qc)

            # add the first m-1 clifford gates
            for j in range(length-1):
                mat = addRandCliff(qc,mat,perm)
            
            #Find String
            genString = findGenString(mat)
            
            print(genString)
            
            # invert everything
            invertCliffGens(qc,genString, perm)
            
            # optional: remove phase
            if phased:
                removePhase(qc,perm)

            # get the qc compiled into one of the 4 different states
            compilation(qc,compiler, perm)
            

            # measure
            for i in range(n):
                qc.measure(qr[perm[i]], cr[i])      
            
            print("got")
            job = execute(qc, backend, shots = shots)
            print("here")
            result = job.result()
            print("nice")
            data = result.get_counts(qc)
            
            # use when running on qx5 and when debugging
            printDataFormatted(data)
            
            # clean the data
            cleanData = {}
            totalSuccessful = 0
            for key in data:
                ohoh = ['0000', '1111']
                ohone = ['1010', '0101']
                oneoh = ['1100', '0011']
                oneone = ['0110', '1001']
                if key not in ohoh+ohone+oneoh+oneone:
                    continue
                if key in ohoh:
                    if "00" not in cleanData.keys():
                        cleanData["00"] = 0
                    cleanData["00"] += data[key]
                    totalSuccessful += data[key]
                elif key in ohone:
                    if "01" not in cleanData.keys():
                        cleanData["01"] = 0
                    cleanData["01"] += data[key]
                    totalSuccessful += data[key]
                elif key in oneoh:
                    if "10" not in cleanData.keys():
                        cleanData["10"] = 0
                    cleanData["10"] += data[key]
                    totalSuccessful += data[key]
                elif key in oneone:
                    if "11" not in cleanData.keys():
                        cleanData["11"] = 0
                    cleanData["11"] += data[key]
                    totalSuccessful += data[key]
            
            # set shouldWrite to false if running on simulator or don't actually care
            if shouldWrite:
                row = writeToExcel(data,genString,compiler,row,col)
            
            # optional: get success rate
            desired = toBinString(compiler)
            print(desired)
            print(cleanData[desired]/totalSuccessful)

endTime = time.time()
print("Time elapsed: ", endTime-startTime)

if shouldWrite:
    workbook.close()


  

Qconfig loaded from C:\Users\ddrsq\OneDrive\Quantum\quantum_rb\Qconfig.py.
3 0

45637
got
here


Got a 403 code response to https://quantumexperience.ng.bluemix.net/api/Jobs?access_token=RQwZSPE9pNNKnyoSfHBjaH4DT7RS7JtPSUvclDy3CG0m8QPqCyXuBSQHQbqSFGrM: {"error":{"status":403,"message":"Your credits are not enough. 3 credits are needed to launch this job but you only have 0 credits remaining.","code":"MAX_CREDITS_EXCEEDED","statusCode":403}}
Got a 403 code response to https://quantumexperience.ng.bluemix.net/api/Jobs?access_token=RQwZSPE9pNNKnyoSfHBjaH4DT7RS7JtPSUvclDy3CG0m8QPqCyXuBSQHQbqSFGrM: {"error":{"status":403,"message":"Your credits are not enough. 3 credits are needed to launch this job but you only have 0 credits remaining.","code":"MAX_CREDITS_EXCEEDED","statusCode":403}}
Got a 403 code response to https://quantumexperience.ng.bluemix.net/api/Jobs?access_token=RQwZSPE9pNNKnyoSfHBjaH4DT7RS7JtPSUvclDy3CG0m8QPqCyXuBSQHQbqSFGrM: {"error":{"status":403,"message":"Your credits are not enough. 3 credits are needed to launch this job but you only have 0 credits remaining.","code

nice


QISKitError: '"{\'status\': 403, \'message\': \'Your credits are not enough. 3 credits are needed to launch this job but you only have 0 credits remaining.\', \'code\': \'MAX_CREDITS_EXCEEDED\', \'statusCode\': 403}"'